Predict on test data

In [ ]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Path to the saved weights
weights_path = '/path/to/trained/model.pt'

# Initialize the YOLO model with the saved weights for instance segmentation
model = YOLO(weights_path, task='segment')

# Path to the test image folder
test_image_folder = '/path/sereact_instance/yolov8_instance/yolo_dataset/test/images/'


test_image_paths = [os.path.join(test_image_folder, fname) for fname in os.listdir(test_image_folder) if fname.endswith(('.jpg', '.jpeg', '.png'))]


for test_image_path in test_image_paths:
    image = cv2.imread(test_image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform inference on the test image
    results = model.predict(image_rgb, conf=0.25)

    # Visualize the results
    plt.figure(figsize=(10, 10))
    plt.imshow(results[0].plot())
    plt.title(os.path.basename(test_image_path))
    plt.axis('off')
    plt.show()



Model evaluation

In [1]:
import os
from ultralytics import YOLO

# Path to the saved weights 
weights_path = '/path/to/trained/model.pt'

# Path to your data.yaml file 
data_yaml_path = '/path/sshete/sereact_instance/yolov8_instance/yolo_dataset/data.yaml'

# Initialize the YOLO model with the saved weights
model = YOLO(weights_path)

# Perform validation using the data.yaml file
results = model.val(data=data_yaml_path)

print(results)


Ultralytics YOLOv8.2.38 🚀 Python-3.8.10 torch-2.0.1+cu118 CUDA:0 (NVIDIA T600 Laptop GPU, 3906MiB)
YOLOv8s-seg summary (fused): 195 layers, 11814817 parameters, 0 gradients, 42.6 GFLOPs


val: Scanning /home/dfki.uni-bremen.de/sshete/sereact_instance/yolov8_instance/yolo_dataset/valid/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


                   all         20        191      0.741      0.389      0.515       0.41      0.761      0.364      0.489      0.353
             aeroplane          2          2      0.784        0.5      0.503      0.403      0.857        0.5      0.503      0.352
                   bag          2          2          1          0      0.995      0.374          1          0      0.995      0.622
            bird_house          2          4      0.548       0.75      0.688      0.619       0.56       0.75      0.717      0.529
                   box          8          9      0.738      0.333      0.399      0.314          1      0.213      0.399      0.341
                   car          3          9          1      0.512      0.946      0.776          1      0.498      0.946      0.719
             container          4          7       0.41      0.117      0.237      0.203      0.344     0.0983      0.237      0.102
            controller          4          4      0.969        0.5   

Convert model to ONX format

In [ ]:
from ultralytics import YOLO

# Path to your saved trained model
custom_model_path = '/path/to/trained/model.pt' 

# Load the custom-trained YOLO model for segmentation
model = YOLO(custom_model_path)

# Export the custom model to ONNX format
model.export(format="onnx")

print("Custom-trained model has been successfully exported to ONNX format.")


Inference comaprision between Yolo model and YOLO in ONNX format

In [6]:
import time
import onnxruntime as ort
import numpy as np
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Paths
weights_path = '/path/weights/best.pt'
onnx_model_path = '/path/yolov8_sereact/results_seg/segment_1st/train/weights/best.onnx'
test_image_path = '/path/sereact_instance/yolov8_sereact/yolo_dataset/train/images/image_6_jpg.rf.ff47b86fe393260073621a6fc5b0a5c0.jpg'


image = cv2.imread(test_image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Preprocess the image for the ONNX model
input_image = cv2.resize(image_rgb, (640, 640))  
input_image = input_image.astype(np.float32) / 255.0  
input_image = np.transpose(input_image, (2, 0, 1))  
input_image = np.expand_dims(input_image, axis=0) 

# Load the YOLO model
yolo_model = YOLO(weights_path)

# Load the ONNX model
ort_session = ort.InferenceSession(onnx_model_path)

# Measure prediction time for YOLO model
start_time = time.time()
yolo_results = yolo_model.predict(image_rgb)
end_time = time.time()
yolo_prediction_time = end_time - start_time

# Measure prediction time for ONNX model
input_name = ort_session.get_inputs()[0].name
start_time = time.time()
onnx_results = ort_session.run(None, {input_name: input_image})
end_time = time.time()
onnx_prediction_time = end_time - start_time

print(f"YOLO model prediction time: {yolo_prediction_time:.4f} seconds")
print(f"ONNX model prediction time: {onnx_prediction_time:.4f} seconds")





0: 640x640 7 bins, 1 car, 1 gun, 1 jar, 408.3ms
Speed: 1.9ms preprocess, 408.3ms inference, 18.6ms postprocess per image at shape (1, 3, 640, 640)
YOLO model prediction time: 1.7023 seconds
ONNX model prediction time: 0.1182 seconds
